## Imports

In [39]:
import pandas as pd
import numpy as np
import os
import sys
import datetime
from os.path import expanduser

### Add to sys.path

Within the barchart_api module, references like:

```from barchartapi import barchart_api```

will not work

In [40]:
WORKSPACE_DIR = '../..' 
PROJECT_DIR = f'{WORKSPACE_DIR}/barchartapi' 
WORKING_DIR = f'{PROJECT_DIR}/barchartapi' 
if WORKING_DIR not in sys.path:
    sys.path.append(WORKING_DIR)
if PROJECT_DIR not in sys.path:
    sys.path.append(PROJECT_DIR)
if WORKSPACE_DIR not in sys.path:
    sys.path.append(WORKSPACE_DIR)
import barchart_api as bcapi

['', '/Library/Frameworks/Python.framework/Versions/3.6/lib/python36.zip', '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6', '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/lib-dynload', '/Users/bperlman1/Virtualenvs3/pyliverisk/lib/python3.6/site-packages', '/Users/bperlman1/Virtualenvs3/pyliverisk/lib/python3.6/site-packages/IPython/extensions', '/Users/bperlman1/.ipython', '../../barchartapi/barchartapi', '../../barchartapi', '../..']


### Create an instance of BcHist

### Create paths to where you will store the history data

In [41]:
# DROPBOX_MARKET_DATA_DIR = expanduser('~/Dropbox/market_data')
# STOCKS_DIR = f'{DROPBOX_MARKET_DATA_DIR}/stocks'
# !ls {STOCKS_DIR}
STOCKS_DIR = f'./temp_folder/stocks'


In [42]:
api_key = open('./temp_folder/paid_api_key.txt','r').read()
bar_type='minutes'
interval=30
endpoint_type='paid_url'
bch = bcapi.BcHist(api_key, bar_type=bar_type, interval=interval,endpoint_type = endpoint_type)


Barchart OnDemand Client: http://ondemand.websol.barchart.com/


## Main Loop to fetch history data from Barchart

In [43]:

spydr_short_names = ['XLE','XLU','XLK','XLB','XLP','XLY','XLI','XLC','XLV','XLF']
commodity_etf_short_names = ['USO','UNG','DBC','DBA','GLD','USCI']
currency_etf_short_names = ['FXY','FXE','FXB','FXF','FXC','FXA']
my_portfolio_short_names = ['XLE','SPY','USO','GLD','XLU']
short_names = my_portfolio_short_names
# beg_yyyymmdd = 20080101
beg_yyyymmdd = 20181101
end_yyyymmdd = 20181205

for short_name in short_names:
    sn_path = f'{STOCKS_DIR}/{short_name}.csv'
    if os.path.isfile(sn_path):
        print(f'BYPASSING: {short_name}')
        continue
    print(f'get_history: {short_name} BEGIN {datetime.datetime.now()}')
    tup = bch.get_history(short_name, beg_yyyymmdd, end_yyyymmdd)
    print(f'get_history: {short_name} WRITING DATA {datetime.datetime.now()}')
    tup[1].to_csv(sn_path,index=False)
    print(f'get_history: {short_name} END {datetime.datetime.now()}')
    

get_history: XLE BEGIN 2019-01-05 09:03:02.243798
get_history: XLE WRITING DATA 2019-01-05 09:03:02.581890
get_history: XLE END 2019-01-05 09:03:02.590245
get_history: SPY BEGIN 2019-01-05 09:03:02.590887
get_history: SPY WRITING DATA 2019-01-05 09:03:02.832313
get_history: SPY END 2019-01-05 09:03:02.840521
get_history: USO BEGIN 2019-01-05 09:03:02.841262
get_history: USO WRITING DATA 2019-01-05 09:03:03.127127
get_history: USO END 2019-01-05 09:03:03.132384
get_history: GLD BEGIN 2019-01-05 09:03:03.132458
get_history: GLD WRITING DATA 2019-01-05 09:03:03.433722
get_history: GLD END 2019-01-05 09:03:03.439608
get_history: XLU BEGIN 2019-01-05 09:03:03.439848
get_history: XLU WRITING DATA 2019-01-05 09:03:03.739494
get_history: XLU END 2019-01-05 09:03:03.745905


In [44]:
df_portfolio = None
for short_name in short_names:
    sn_path = f'{STOCKS_DIR}/{short_name}.csv'
    df_temp = pd.read_csv(sn_path)
    df_temp = df_temp[['timestamp','close']]
    df_temp = df_temp.rename(columns={'close':f'{short_name}'})
    if df_portfolio is None:
        df_portfolio = df_temp.copy()
    else:
        df_portfolio = df_portfolio.merge(df_temp,how='inner',on='timestamp')
df_corr = df_portfolio.corr()
df_corr

,XLE,SPY,USO,GLD,XLU
XLE,1.000000,0.975327,0.901554,-0.821944,0.368654
SPY,0.975327,1.000000,0.825213,-0.829579,0.456716
USO,0.901554,0.825213,1.000000,-0.708552,0.151254
GLD,-0.821944,-0.829579,-0.708552,1.000000,-0.419358
XLU,0.368654,0.456716,0.151254,-0.419358,1.000000


In [45]:
fn = f'{STOCKS_DIR}/{"_".join(df_corr.columns.values)}.csv'
df_corr.to_csv(fn,index=False)

In [46]:
df_portfolio.std()

XLE     3.997376
SPY    11.089229
USO     1.198228
GLD     2.203888
XLU     1.282801
dtype: float64

## END